In [1]:
import pydra
from pydra.tasks import fsl
from pathlib import Path
from ipyniivue import Niivue
import shutil
import pydra
import os
import nest_asyncio
import shutil
nest_asyncio.apply()
deface_state = False
from pathlib import Path
import os

import shutil
import nibabel as nib
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [7]:
patientdir = Path("/Users/rosana.eljurdi/Datasets/err/Lymphoma_oriented2")
labeldir = Path(patientdir, "manual_segm")


label_paths = list(sorted(labeldir.rglob('*.nii.gz')))
image_paths = list(sorted(set(patientdir.rglob('*.nii.gz')) - set(label_paths)))

image_paths


[PosixPath('/Users/rosana.eljurdi/Datasets/err/Lymphoma_oriented2/sub-1075635/ses-M000/anat/sub-1075635_ses-M000_T1w.nii.gz')]

In [8]:
def deface_and_resample_to_template(**kwargs) -> pydra.Workflow:
    from pydra.tasks import fsl
    from pydra.tasks.fsl import fslmaths

    workflow = pydra.Workflow(
        input_spec=[
            "input_image",
            "template_image",
            "template_mask",
            "segm_image"
        ],
        **kwargs,
    )

    workflow.add(
        fsl.FSLReorient2Std(
            name="fslreorient2std",
            input_image=workflow.lzin.input_image,
        )
    )

    reference_image = (
        workflow.lzin.template_image
    )

    workflow.add(
        fsl.FLIRT(
            name="flirt",
            input_image=workflow.fslreorient2std.lzout.output_image,
            reference_image=workflow.lzin.template_image,
            cost_function="mutualinfo",
        )
    )

    workflow.add(
        fslmaths.Mul(
            name="apply_mask",
            input_image=workflow.flirt.lzout.output_image,
            other_image=workflow.lzin.template_mask,
        )
    )
    
    workflow.add(
        fsl.FSLReorient2Std(
            name="fslreorient2stdsegm",
            input_image=workflow.lzin.segm_image,
        )
    )
    
    workflow.add(
        fsl.ApplyXFM(
            name="apply_xfm",
            input_image=workflow.fslreorient2stdsegm.lzout.output_image,
            reference_image=reference_image,
            input_matrix=workflow.flirt.lzout.output_matrix,
            interpolation="nearestneighbour",
        )
    )
    


    workflow.set_output({
        "output_image": workflow.apply_mask.lzout.output_image,
        "registered_GT": workflow.apply_xfm.lzout.output_image,
        "output_matric": workflow.flirt.lzout.output_matrix
    })

    return workflow


In [9]:
for image_path in image_paths:
    new_path = Path(str(image_path.parent).replace('Lymphoma_oriented2', 'Lymphoma-defaced-brain-mni'))
    new_path.mkdir(parents=True, exist_ok=True)
    print(new_path)
    
    
for label_path in label_paths:
    new_path = Path(str(label_path.parent).replace('Lymphoma_oriented2', 'Lymphoma-defaced-brain-mni'))
    new_path.mkdir(parents=True, exist_ok=True)
    
    print(new_path)

/Users/rosana.eljurdi/Datasets/err/Lymphoma-defaced-brain-mni/sub-1075635/ses-M000/anat
/Users/rosana.eljurdi/Datasets/err/Lymphoma-defaced-brain-mni/manual_segm/sub-1075635/ses-M000/anat


In [11]:
image_paths

[PosixPath('/Users/rosana.eljurdi/Datasets/err/Lymphoma_oriented2/sub-1075635/ses-M000/anat/sub-1075635_ses-M000_T1w.nii.gz')]

In [12]:
# defacing 



for patient_path in enumerate(zip(image_paths, label_paths)):
    print(patient_path)

    Task = deface_and_resample_to_template(input_image=patient_path[1][0], name='deface', 
              template_image='/Users/rosana.eljurdi/Datasets/defacing-templates/tpl-MNI152Lin_res-01_T1w.nii.gz', 
              template_mask = '/Users/rosana.eljurdi/Datasets/defacing-templates/tpl-MNI152Lin_res-01_desc-brain_mask.nii.gz',
              segm_image = patient_path[1][1]  ,
             )

    result = Task()
    
    
    #shutil.copy(str(result.output.output_image), str(patient_path[1][0]).replace("Lymphoma_oriented2", "Lymphoma-defaced-brain-mni"))
    #shutil.copy(str(result.output.registered_GT), str(patient_path[1][1]).replace("Lymphoma_oriented2", "Lymphoma-defaced-brain-mni"))
    print(str(patient_path).replace("Lymphoma_oriented2", "Lymphoma-defaced-brain-mni"))
    print(result.output)
    print(patient_path)

(0, (PosixPath('/Users/rosana.eljurdi/Datasets/err/Lymphoma_oriented2/sub-1075635/ses-M000/anat/sub-1075635_ses-M000_T1w.nii.gz'), PosixPath('/Users/rosana.eljurdi/Datasets/err/Lymphoma_oriented2/manual_segm/sub-1075635/ses-M000/anat/sub-1075635_ses-M000_dseg.nii.gz')))
(0, (PosixPath('/Users/rosana.eljurdi/Datasets/err/Lymphoma-defaced-brain-mni/sub-1075635/ses-M000/anat/sub-1075635_ses-M000_T1w.nii.gz'), PosixPath('/Users/rosana.eljurdi/Datasets/err/Lymphoma-defaced-brain-mni/manual_segm/sub-1075635/ses-M000/anat/sub-1075635_ses-M000_dseg.nii.gz')))
Output(output_image=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmp1y7kv46b/Workflow_47de54392e0c660b72f259f807db39420b0f54c1d0c74c493578bae22ac9a5cf/sub-1075635_ses-M000_T1w_r2std_flirt_fslmaths.nii.gz'), registered_GT=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmp1y7kv46b/Workflow_47de54392e0c660b72f259f807db39420b0f54c1d0c74c493578bae22ac9a5cf/sub-1075635_ses-M000_dseg_r2std_axfm.nii.gz'

In [10]:
patient_path = "/Users/rosana.eljurdi/Datasets/err/Lymphoma_oriented2/sub-1071135/ses-M000/anat/sub-1071135_ses-M000_T1w.nii.gz"

segm_path = "/Users/rosana.eljurdi/Datasets/err/Lymphoma_oriented2/manual_segm/sub-1071135/ses-M000/anat/sub-1071135_ses-M000_dseg.nii.gz"

Task = deface_and_resample_to_template(input_image=patient_path, name='deface', 
          template_image='/Users/rosana.eljurdi/Datasets/defacing-templates/tpl-MNI152Lin_res-01_T1w.nii.gz', 
          template_mask = '/Users/rosana.eljurdi/Datasets/defacing-templates/tpl-MNI152Lin_res-01_desc-brain_mask.nii.gz',
          segm_image = segm_path  ,
         )

result = Task()


#shutil.copy(str(result.output.output_image), str(patient_path[1][0]).replace("Lymphoma", "Lymphoma-defaced-brain-mni"))
#shutil.copy(str(result.output.registered_GT), str(patient_path[1][1]).replace("Lymphoma", "Lymphoma-defaced-brain-mni"))
#print(str(patient_path).replace("Lymphoma", "Lymphoma-defaced-brain-mni"))
print(result.output)
print(patient_path)
    

Output(output_image=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmp1t55u3zc/Workflow_6cec02d9ef5eeea30dc1b84e40f7b45d61b45cbbb87bb9a6dfdb89451aaa853e/sub-1071135_ses-M000_T1w_r2std_flirt_fslmaths.nii.gz'), registered_GT=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmp1t55u3zc/Workflow_6cec02d9ef5eeea30dc1b84e40f7b45d61b45cbbb87bb9a6dfdb89451aaa853e/sub-1071135_ses-M000_dseg_r2std_axfm.nii.gz'), output_matric=PosixPath('/private/var/folders/vl/8gp_6lm16ksg3bnh7__sjzwc00102x/T/tmp1t55u3zc/Workflow_6cec02d9ef5eeea30dc1b84e40f7b45d61b45cbbb87bb9a6dfdb89451aaa853e/sub-1071135_ses-M000_T1w_r2std_flirt.mat'))
/Users/rosana.eljurdi/Datasets/err/Lymphoma_oriented2/sub-1071135/ses-M000/anat/sub-1071135_ses-M000_T1w.nii.gz


In [ ]:

import nibabel as nb

patientdir = Path("/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni")
labeldir = Path(patientdir, "derivatives/manual_segm")


label_paths = list(sorted(labeldir.rglob('*.nii.gz')))
image_paths = list(sorted(set(patientdir.rglob('*.nii.gz')) - set(label_paths)))


print(len(label_paths))


for label in label_paths:
    nft = nb.load(label).get_fdata()
    print(label.stem, nft.sum(), np.unique(nft) )




In [ ]:

import nibabel as nb

patientdir = Path("/Users/rosana.eljurdi/Datasets/Lymphoma")
labeldir = Path(patientdir, "derivatives/manual_segm")


label_paths = list(sorted(labeldir.rglob('*.nii.gz')))
image_paths = list(sorted(set(patientdir.rglob('*.nii.gz')) - set(label_paths)))


print(len(label_paths))


for label in label_paths:
    nft = nb.load(label).get_fdata()
    print(label.stem, nft.sum(), np.unique(nft) )


In [ ]:

# Sanity check: 

for image_path in enumerate(zip(image_paths, label_paths)):
        img = nib.load(image_path[1][0]).get_fdata()
        gt = nib.load(image_path[1][1]).get_fdata()
        print(os.path.basename(image_path[1][0]), img.shape,gt.shape )
        assert (img.shape == gt.shape)
   

In [ ]:
#Getting the resolution

patientdir = Path("/Users/rosana.eljurdi/Datasets/Lymphoma")
labeldir = Path(patientdir, "derivatives/manual_segm")


label_paths = list(sorted(labeldir.rglob('*.nii.gz')))
image_paths = list(sorted(set(patientdir.rglob('*.nii.gz')) - set(label_paths)))

ppixdim_info = []
for patient_path in image_paths:

    task = fsl.FSLInfo(input_image=patient_path)
    result = task()

    ppixdim_info.append({'subject': patient_path.stem,
                         'ndim1':result.output.pixdim1,
                         'ndim2': result.output.pixdim2,
                         'ndim3': result.output.pixdim3 })


df = pd.DataFrame(data=ppixdim_info)
df.to_csv(os.path.join(patientdir,'resolution.csv'))

In [ ]:
df_AgeSex

In [ ]:
Meta_data

In [ ]:

Meta_data

In [ ]:
# getting the partitions


In [ ]:
Meta_data.to_csv("/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/metadata.csv")
